## Modele de clasificare
Folositi 4 seturi de date pentru probleme de clasificare, plecand de la repository-urile specificate in Cursul 6. Cel putin un set de date sa fie cu valori lipsa; pentru un alt set de date care are initial toate valorile, introduceti dvs. in mod artificial valori lipsa, suprascriind un anumit procent din valorile initiale (ex. p=5%, p parametru) cu numpy.nan.

In [162]:
import numpy as np
print(f'NumPy version: {np.__version__}')

import sklearn as sk
from sklearn.impute import SimpleImputer
print(f'Sklearn version: {sk.__version__}')

#NumPy version: 1.20.1
#Sklearn version: 0.24.1

NumPy version: 1.20.1
Sklearn version: 0.24.1


In [151]:
wine:np.array=np.genfromtxt("Datasets/wine.data", delimiter=",")
iris:np.array=np.genfromtxt("Datasets/iris.data", delimiter=",")
machine:np.array=np.genfromtxt("Datasets/machine.data", delimiter=",")
wdbc:np.array=np.genfromtxt("Datasets/wdbc.data", delimiter=",")

1. (20 puncte) Aplicati o metoda de missing value imputation, unde este cazul; documentati metoda folosita.
*Resurse*: Pentru missing value imputation, puteti urmari [Imputation of missing values](https://scikit-learn.org/stable/modules/impute.html), [How to Handle Missing Data with Python](https://machinelearningmastery.com/handle-missing-data-python/), [fancyimpute](https://github.com/iskandr/fancyimpute), [missingpy](https://github.com/epsilon-machine/missingpy).

*Cerinta*: In cazul in care folositi un pachet ce trebuie instalat (nu face parte din distributia standard anaconda), includeti intr-o celula o comanda de instalare corespunzatoare folosind semn de exclamare, de exemplu:
```python
!pip install missingpy
```
(sursa: [https://github.com/epsilon-machine/missingpy](https://github.com/epsilon-machine/missingpy)). La executia celulei in Jupyter Notebook se instaleaza pachetul, iar in celulele ulterioare importurile din noul pachet functioneaza. 

In [152]:
def Random_Nans(data_set:np.array,percentage:int = 5)->np.array:
    """
    Adding NaN values in a dataset.
    Args:
        data_set:the numpy.ndarray containing the dataset
        percentage:the percentage(from dataset size) of Nans values to be added in the dataset
    Returns:the modified dataset
    """
    result_set:np.array=data_set.copy()
    choice:np.array = np.random.choice(result_set.size, int(percentage / 100 * result_set.size), replace=False)
    rows:np.array, cols:np.array = np.unravel_index(choice, result_set.shape)
    result_set[rows, cols] = np.nan
    return result_set

In [155]:
wine_test:np.array=Random_Nans(wine)
iris_test:np.array=Random_Nans(iris)
machine_test:np.array=Random_Nans(machine)
wdbc_test:np.array=Random_Nans(iris_test)

In [156]:
imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
wine_test:np.array=imp.fit_transform(wine_test)
iris_test:np.array=imp.fit_transform(iris_test)
machine_test:np.array=imp.fit_transform(machine_test)
wdbc_test:np.array=imp.fit_transform(wdbc_test)